In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [3]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
import qiskit.providers.aer.noise as noise
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts
from vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits, organize_tomography_data_from_list

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.3.0', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [5]:
numpy.set_printoptions(linewidth = 200)

qubits = 5

simulation_backend = "qasm_simulator"

seed = 0
print_circuits = True

In [6]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [9]:
device = provider.get_backend('ibmq_toronto')
device

<IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [10]:
device.configuration().to_dict()['max_shots']

32000

In [11]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [12]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [13]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_front.qasm")

In [14]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_mid.qasm")

In [15]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_end.qasm")

In [16]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69769/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [17]:
orign_circuit.draw()

┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────────────■────────»
     ├───────────────┤                                              │        »
q_3: ┤ Ry(-0.019851) ├──────────────────────────────────────────────■────────»
     └───────────────┘                                                       »
q_4: ────────────────────────────────────────────────────────────────────────»
                                                                             »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«     ┌───────────────┐
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘
«q_4: ─────────────────
«

In [18]:
#checking circuit

In [19]:
def apply_checking_circuit(qc, ctrl_bits, ancilla_bits, side = None):
    if len(ctrl_bits) != len(ancilla_bits):
        print("Size mismatch")
        return None
    if side == 'front':
        for i in ancilla_bits:
            qc.h(i)
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
    elif side == 'end':
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
        for i in ancilla_bits:
            qc.h(i)
    else:
        print("Side undefined")

In [20]:
circuit = front_layer.copy()

In [21]:
apply_checking_circuit(circuit, [3], [4], side = 'front')
circuit += mid_layer
apply_checking_circuit(circuit, [3], [4], side = 'end')
circuit += end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69769/4156076483.py:2: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += mid_layer
/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69769/4156076483.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += end_layer


In [22]:
circuit.draw()

┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────────────■────────»
     ├───────────────┤                                              │        »
q_3: ┤ Ry(-0.019851) ├────────■─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                                      
«q_0: ─────────────────────────────────
«                                      
«q_1: ─────────────────────────────────
«     ┌───────────────┐                
«q_2: ┤ Ry(0.0036205) ├────────────────
«     └───────────────┘┌──────────────┐
«q_3: ────────■────────┤ Ry(0.019873) ├
«             │        └────┬───┬─────┘
«q_4: ────────■─────────────┤ H ├──────
«                           └───┘

In [23]:
pauli_list = read_from_file('LiH_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [24]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 4,num_qargs = 5)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YYYY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
                     
q_3: ────────────────
                     
q_4: ────────────────
                     
ZZYY
                     
q_0: ────────────────
                     
q_1: ────────────────
     ┌──────────────┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZX
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
      ┌────────────┐ 
q_3: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_4: ────────────────
                     
ZXYY
       

In [25]:
circuit_list = []
for meas in meas_qcs:
    temp_circuit = circuit.copy()
    temp_circuit += meas
    circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69769/1717817760.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [26]:
circuit_list[-2].draw()

┌───┐      ┌───────────────┐   ┌────────────────┐  ┌────────────┐ »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├──┤ U(π/2,0,π) ├─»
           ├───┤      ├───────────────┤ │ └────────────────┘┌─┴────────────┴┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────────────■────────»
     ├───────────────┤                                              │        »
q_3: ┤ Ry(-0.019851) ├────────■─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                                                    
«q_0: ───────────────────────────────────────────────
«       ┌────────────┐                               
«q_1: ──┤ U(π/2,0,π) ├───────────────────────────────
«     ┌─┴────────────┴┐ ┌────────────┐               
«q_2: ┤ Ry(0.0036205) ├─┤ U(π/2,0,π) ├───────────────
«     └───────────────┘┌┴────────────┴┐┌────────────┐
«q_3: ────────■────────┤ Ry(0.019873) ├┤ U(π/2,0,π) ├
«             │        └────┬───┬─────┘└────────────┘
«q_4: ────────■─────────────┤ H ├────────────────────
«                           └───┘

In [27]:
# cuts = []
# for i in range(0, 1):
#     if i == 0 or i == qubits - 1:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 6)]
#     else:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 4)]
#     cuts += cut

In [28]:
cuts = [(circuit.qubits[3], 2),(circuit.qubits[3], 3)]

In [29]:
fragments_list = []
wire_path_map_list = []
for temp_qc in circuit_list:
    print(temp_qc)
    fragments, wire_path_map = circuit_cutter.cut_circuit(temp_qc, cuts)
    fragments_list.append(fragments)
    wire_path_map_list.append(wire_path_map)

           ┌───┐      ┌───────────────┐   ┌────────────────┐ ┌──────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┤ U(π/2,0,π/2) ├»
           ├───┤      ├───────────────┤ │ └────────────────┘┌┴──────────────┤»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────────────■────────»
     ├───────────────┤                                              │        »
q_3: ┤ Ry(-0.019851) ├────────■─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                                                      
«q_0: ─────────────────────────────────────────────────
«      ┌───────────

           ┌───┐      ┌───────────────┐   ┌────────────────┐ ┌──────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┤ U(π/2,0,π/2) ├»
           ├───┤      ├───────────────┤ │ └────────────────┘┌┴──────────────┤»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────────────■────────»
     ├───────────────┤                                              │        »
q_3: ┤ Ry(-0.019851) ├────────■─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                                      
«q_0: ─────────────────────────────────
«      ┌──────────────┐                
«q_1: ─┤ U(

           ┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────────────■────────»
     ├───────────────┤                                              │        »
q_3: ┤ Ry(-0.019851) ├────────■─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                                      
«q_0: ─────────────────────────────────
«       ┌────────────┐                 
«q_1: ──┤ U

           ┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────────────■────────»
     ├───────────────┤                                              │        »
q_3: ┤ Ry(-0.019851) ├────────■─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                                                    
«q_0: ───────────────────────────────────────────────
«       ┌────────────┐ 

In [30]:
print(wire_path_map_list[0])

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(4, 'q'), 1)),), Qubit(QuantumRegister(5, 'q'), 1): ((0, Qubit(QuantumRegister(4, 'q'), 2)),), Qubit(QuantumRegister(5, 'q'), 2): ((0, Qubit(QuantumRegister(4, 'q'), 0)),), Qubit(QuantumRegister(5, 'q'), 3): ((1, Qubit(QuantumRegister(3, 'q'), 0)), (0, Qubit(QuantumRegister(4, 'q'), 3)), (1, Qubit(QuantumRegister(3, 'q'), 2))), Qubit(QuantumRegister(5, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 1)),)}


In [31]:
total_variants = ml.fragment_variants(wire_path_map_list[0])

In [32]:
total_variants

24

In [33]:
shots = device.configuration().to_dict()['max_shots'] * total_variants

In [34]:
fragments_list[0][0].draw()

┌───────────────┐                                         »
q_0: ┤ Ry(0.0038243) ├───────────────────────────────────────■─»
     └─────┬───┬─────┘┌───────────────┐   ┌────────────────┐ │ »
q_1: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┼─»
           ├───┤      ├───────────────┤ │ └────────────────┘ │ »
q_2: ──────┤ X ├──────┤ Ry(-0.015869) ├─■────────────────────■─»
           └───┘      └───────────────┘                        »
q_3: ──────────────────────────────────────────────────────────»
                                                               »
«                         ┌───────────────┐┌──────────────┐
«q_0: ──────────────────■─┤ Ry(0.0036205) ├┤ U(π/2,0,π/2) ├
«      ┌──────────────┐ │ └───────────────┘└──────────────┘
«q_1: ─┤ U(π/2,0,π/2) ├─┼──────────────────────────────────
«     ┌┴──────────────┤ │  ┌──────────────┐                
«q_2: ┤ Ry(-0.015751) ├─┼──┤ U(π/2,0,π/2) ├────────────────
«     └───────────────┘ │  └──────────────┘                
«q_3: ──────────────────■──────────────────────────────────
«

In [35]:
fragment_cuts_list = []
for index in range(0, len(fragments_list)):
    for idx, fragment in enumerate(fragments_list[index]):
        print(f"fragment {idx}:")
        print(fragment)
        print()
    fragment_cuts = ml.fragment_cuts(wire_path_map_list[index])
    fragment_cuts_list.append(fragment_cuts)


fragment 0:
     ┌───────────────┐                                         »
q_0: ┤ Ry(0.0038243) ├───────────────────────────────────────■─»
     └─────┬───┬─────┘┌───────────────┐   ┌────────────────┐ │ »
q_1: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┼─»
           ├───┤      ├───────────────┤ │ └────────────────┘ │ »
q_2: ──────┤ X ├──────┤ Ry(-0.015869) ├─■────────────────────■─»
           └───┘      └───────────────┘                        »
q_3: ──────────────────────────────────────────────────────────»
                                                               »
«                         ┌───────────────┐┌──────────────┐
«q_0: ──────────────────■─┤ Ry(0.0036205) ├┤ U(π/2,0,π/2) ├
«      ┌──────────────┐ │ └───────────────┘└──────────────┘
«q_1: ─┤ U(π/2,0,π/2) ├─┼──────────────────────────────────
«     ┌┴──────────────┤ │  ┌──────────────┐                
«q_2: ┤ Ry(-0.015751) ├─┼──┤ U(π/2,0,π/2) ├────────────────
«     └───────────────┘ │  └──────────────┘

In [36]:
print(fragment_cuts_list[-1])

[{'prep': 1, 'meas': 1}, {'prep': 1, 'meas': 1}]


In [37]:
wire_path_map

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(4, 'q'), 1)),),
 Qubit(QuantumRegister(5, 'q'), 1): ((0, Qubit(QuantumRegister(4, 'q'), 2)),),
 Qubit(QuantumRegister(5, 'q'), 2): ((0, Qubit(QuantumRegister(4, 'q'), 0)),),
 Qubit(QuantumRegister(5, 'q'), 3): ((1, Qubit(QuantumRegister(3, 'q'), 0)),
  (0, Qubit(QuantumRegister(4, 'q'), 3)),
  (1, Qubit(QuantumRegister(3, 'q'), 2))),
 Qubit(QuantumRegister(5, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 1)),)}

In [38]:
num_fragments = len(fragments)
num_fragments

2

In [39]:
num_tomo_circuits = len(meas_qcs)
num_tomo_circuits

25

In [40]:
fragments[0].draw()

┌───────────────┐                                         »
q_0: ┤ Ry(0.0038243) ├───────────────────────────────────────■─»
     └─────┬───┬─────┘┌───────────────┐   ┌────────────────┐ │ »
q_1: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┼─»
           ├───┤      ├───────────────┤ │ └────────────────┘ │ »
q_2: ──────┤ X ├──────┤ Ry(-0.015869) ├─■────────────────────■─»
           └───┘      └───────────────┘                        »
q_3: ──────────────────────────────────────────────────────────»
                                                               »
«                         ┌───────────────┐
«q_0: ──────────────────■─┤ Ry(0.0036205) ├
«                       │ └───────────────┘
«q_1: ──────────────────┼──────────────────
«     ┌───────────────┐ │                  
«q_2: ┤ Ry(-0.015751) ├─┼──────────────────
«     └───────────────┘ │                  
«q_3: ──────────────────■──────────────────
«

In [41]:
fragments[1].draw()

┌───────────────┐                      
q_0: ┤ Ry(-0.019851) ├─■────────────────────
     └─────┬───┬─────┘ │         ┌───┐      
q_1: ──────┤ H ├───────■──■──────┤ H ├──────
           └───┘          │ ┌────┴───┴─────┐
q_2: ─────────────────────■─┤ Ry(0.019873) ├
                            └──────────────┘

In [42]:
hardware_index = 0

In [43]:
tomography_circuits = []
for idx in range(0, len(fragments_list)):
    fragments = fragments_list[idx]
    wire_path_map = wire_path_map_list[idx]
    temp_frag_circuits = [0] * len(fragments)
    for i in range(0, len(fragments)):
        if i == hardware_index:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend, extra_qc = None)
        else:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
    tomography_circuits.append(temp_frag_circuits)

In [44]:
#contains 25 basis
len(tomography_circuits)

25

In [45]:
#contains three fragments
len(tomography_circuits[0])

2

In [46]:
len(tomography_circuits[0][1])

1

In [47]:
#tomography circuits
len(tomography_circuits[0][1][0])

12

In [48]:
multi_exp_circuits = [[],[]]
for i in range(0, len(tomography_circuits)):
    #for each basis collect the three fragments:
    for j in range(0, num_fragments):
        fragment_circ = tomography_circuits[i][j][0]
        multi_exp_circuits[j] += fragment_circ

In [49]:
multi_exp_circuits[0][-200].draw()

░ ┌───────────────┐                    »
  q_0: ────────────────────░─┤ Ry(0.0038243) ├────────────────────»
                           ░ └─────┬───┬─────┘┌───────────────┐   »
  q_1: ────────────────────░───────┤ X ├──────┤ Ry(-0.003534) ├─■─»
                           ░       ├───┤      ├───────────────┤ │ »
  q_2: ────────────────────░───────┤ X ├──────┤ Ry(-0.015869) ├─■─»
       ┌─────────────────┐ ░       └───┘      └───────────────┘   »
  q_3: ┤ U3(-1.9106,π,0) ├─░──────────────────────────────────────»
       └─────────────────┘ ░                                      »
c16_0: ═══════════════════════════════════════════════════════════»
                                                                  »
c16_1: ═══════════════════════════════════════════════════════════»
                                                                  »
c16_2: ═══════════════════════════════════════════════════════════»
                                                                  »
c16_3: ═══════════════════════════════════════════════════════════»
                                                                  »
«                                                ┌───────────────┐»
«  q_0: ───────────────────■───────────────────■─┤ Ry(0.0036205) ├»
«       ┌────────────────┐ │  ┌──────────────┐ │ └───────────────┘»
«  q_1: ┤ Ry(-0.0036834) ├─┼──┤ U(π/2,0,π/2) ├─┼──────────────────»
«       └────────────────┘ │ ┌┴──────────────┤ │  ┌──────────────┐»
«  q_2: ───────────────────■─┤ Ry(-0.015751) ├─┼──┤ U(π/2,0,π/2) ├»
«                            └───────────────┘ │  └──────────────┘»
«  q_3: ───────────────────────────────────────■──────────────────»
«                                                                 »
«c16_0: ══════════════════════════════════════════════════════════»
«                                                                 »
«c16_1: ══════════════════════════════════════════════════════════»
«                                                                 »
«c16_2: ══════════════════════════════════════════════════════════»
«                                                                 »
«c16_3: ══════════════════════════════════════════════════════════»
«                                                                 »
«       ┌────────────┐ ░ ┌─┐              
«  q_0: ┤ U(π/2,0,π) ├─░─┤M├──────────────
«       └────────────┘ ░ └╥┘┌─┐           
«  q_1: ───────────────░──╫─┤M├───────────
«                      ░  ║ └╥┘┌─┐        
«  q_2: ───────────────░──╫──╫─┤M├────────
«                      ░  ║  ║ └╥┘┌───┐┌─┐
«  q_3: ───────────────░──╫──╫──╫─┤ H ├┤M├
«                      ░  ║  ║  ║ └───┘└╥┘
«c16_0: ══════════════════╩══╬══╬═══════╬═
«                            ║  ║       ║ 
«c16_1: ═════════════════════╩══╬═══════╬═
«                               ║       ║ 
«c16_2: ════════════════════════╩═══════╬═
«                                       ║ 
«c16_3: ════════════════════════════════╩═
«

In [50]:
frag_datas = [0] * num_fragments
for i in range(0, num_fragments):
    if i == hardware_index:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = [11,16,13,8], 
                                     backend = device, shots = shots // total_variants, 
                                     optimization_level = 3, monitor_jobs = True)
    else:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = None,
                                     backend = simulation_backend, shots = shots // total_variants,
                                     optimization_level = 3, monitor_jobs = True, noise_model = None)
       

job_id:  63777fe618d7487bf8f75528
Job Status: job is queued (16)    

IBMQJobApiError: '"HTTPSConnectionPool(host=\'api-qcon.quantum-computing.ibm.com\', port=443): Max retries exceeded with url: /api/Network/ibm-q-ornl/Groups/ornl/Projects/chm185/Jobs/63777fe618d7487bf8f75528/status/v/1 (Caused by NewConnectionError(\'<urllib3.connection.HTTPSConnection object at 0x7fd469ce8ac0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known\'))"'

In [51]:
frag_datas[0] = [device.retrieve_job(job_id='63777fe618d7487bf8f75528').result()]

In [52]:
frag_datas[1] = run_circuits(multi_exp_circuits[1], initial_layout = None,
                             backend = simulation_backend, shots = shots // total_variants,
                             optimization_level = 3, monitor_jobs = True, noise_model = None)

job_id:  3e9e4732-1b12-40e9-9072-70cbaeb8e61d
Job Status: job has successfully run


In [53]:
frag_datas[0][0].results[0].header.name

"(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z'))"

In [54]:
num_tomo_circuits

25

In [55]:
test = [[], []]

In [56]:
results_list = [[], []]
counts_list = [[], []] 
for i in range(0, num_fragments): 
    for j in range(0, num_tomo_circuits):
        start_idx = j * 12
        end_idx = (j + 1) * 12
        results_list[i].append(frag_datas[i][0].results[start_idx : end_idx])
        counts_list[i].append(frag_datas[i][0].get_counts()[start_idx : end_idx])


In [57]:
frag_datas[0][0].get_counts()

[{'0000': 4561,
  '0001': 4355,
  '0010': 4291,
  '0011': 3990,
  '0100': 3874,
  '0101': 3725,
  '0110': 3590,
  '0111': 3285,
  '1000': 56,
  '1001': 48,
  '1010': 51,
  '1011': 26,
  '1100': 47,
  '1101': 36,
  '1110': 29,
  '1111': 36},
 {'0000': 2623,
  '0001': 2353,
  '0010': 2466,
  '0011': 2221,
  '0100': 2090,
  '0101': 1995,
  '0110': 1833,
  '0111': 1807,
  '1000': 2073,
  '1001': 2022,
  '1010': 1888,
  '1011': 1912,
  '1100': 1738,
  '1101': 1814,
  '1110': 1581,
  '1111': 1584},
 {'0000': 2311,
  '0001': 1995,
  '0010': 2213,
  '0011': 1890,
  '0100': 1819,
  '0101': 1929,
  '0110': 1747,
  '0111': 1651,
  '1000': 2324,
  '1001': 2363,
  '1010': 2131,
  '1011': 2237,
  '1100': 2003,
  '1101': 1845,
  '1110': 1893,
  '1111': 1649},
 {'0000': 1598,
  '0001': 1542,
  '0010': 1595,
  '0011': 1516,
  '0100': 1327,
  '0101': 1184,
  '0110': 1293,
  '0111': 1262,
  '1000': 2799,
  '1001': 3077,
  '1010': 2742,
  '1011': 3115,
  '1100': 2111,
  '1101': 2352,
  '1110': 2139,
  '11

In [58]:
frag_datas[0][0].get_counts()

[{'0000': 4561,
  '0001': 4355,
  '0010': 4291,
  '0011': 3990,
  '0100': 3874,
  '0101': 3725,
  '0110': 3590,
  '0111': 3285,
  '1000': 56,
  '1001': 48,
  '1010': 51,
  '1011': 26,
  '1100': 47,
  '1101': 36,
  '1110': 29,
  '1111': 36},
 {'0000': 2623,
  '0001': 2353,
  '0010': 2466,
  '0011': 2221,
  '0100': 2090,
  '0101': 1995,
  '0110': 1833,
  '0111': 1807,
  '1000': 2073,
  '1001': 2022,
  '1010': 1888,
  '1011': 1912,
  '1100': 1738,
  '1101': 1814,
  '1110': 1581,
  '1111': 1584},
 {'0000': 2311,
  '0001': 1995,
  '0010': 2213,
  '0011': 1890,
  '0100': 1819,
  '0101': 1929,
  '0110': 1747,
  '0111': 1651,
  '1000': 2324,
  '1001': 2363,
  '1010': 2131,
  '1011': 2237,
  '1100': 2003,
  '1101': 1845,
  '1110': 1893,
  '1111': 1649},
 {'0000': 1598,
  '0001': 1542,
  '0010': 1595,
  '0011': 1516,
  '0100': 1327,
  '0101': 1184,
  '0110': 1293,
  '0111': 1262,
  '1000': 2799,
  '1001': 3077,
  '1010': 2742,
  '1011': 3115,
  '1100': 2111,
  '1101': 2352,
  '1110': 2139,
  '11

In [59]:
from mlrecon_methods import organize_tomography_data_from_list
organized_data = []
for i in range(0, num_tomo_circuits):
    frag_data = []
    frag_targets = ml.identify_frag_targets(wire_path_map_list[i])
    for j in range(0, num_fragments):
        frag_data.append(organize_tomography_data_from_list(results_list[j][i], counts_list[j][i],
                                              frag_targets[j].get("prep"),
                                              frag_targets[j].get("meas"),
                                              prep_basis = "SIC", extra_qc = []))
    organized_data.append(frag_data)

(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S0',) ('Z',) [3] [3]
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'X')) ('S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'X')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'X')) ('S0',) ('X',) [3] [3]
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Y')) ('S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Y')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Y')) ('S0',) ('Y',) [3] [3]
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S1') ('Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'Z')) ('S1',) ('Z',) [3] [3]
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'X')) ('S0', 'S0', 'S0', 'S1') ('Z', 'Z', 'Z', 'X')
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'X')) ('S1',) ('X',) [3] [3]
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'Y')) ('S0', 'S0', 'S0', 'S1') ('Z', 'Z', 'Z', 'Y')
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'Y')) ('S1',) ('Y',) [3] [3

In [60]:
for final_bits, fixed_bit_data in frag_data[0].items():
    prep_meas_states, state_counts = zip(*fixed_bit_data.items())
    prep_labels, meas_labels = zip(*prep_meas_states)
    prep_qubit_num = len(prep_labels[0])
    meas_qubit_num = len(meas_labels[0])
    print(prep_qubit_num)

1
1
1
1
1
1
1
1


In [61]:
len(organized_data)

25

In [62]:
organized_data[2]

[{'000': {(('S0',), ('Zp',)): 23,
   (('S0',), ('Xp',)): 17,
   (('S0',), ('Xm',)): 12,
   (('S0',), ('Yp',)): 22,
   (('S0',), ('Ym',)): 21,
   (('S1',), ('Zp',)): 12,
   (('S1',), ('Zm',)): 20,
   (('S1',), ('Xp',)): 40,
   (('S1',), ('Xm',)): 3,
   (('S1',), ('Yp',)): 25,
   (('S1',), ('Ym',)): 16,
   (('S2',), ('Zp',)): 20,
   (('S2',), ('Zm',)): 20,
   (('S2',), ('Xp',)): 13,
   (('S2',), ('Xm',)): 23,
   (('S2',), ('Yp',)): 2,
   (('S2',), ('Ym',)): 28,
   (('S3',), ('Zp',)): 4,
   (('S3',), ('Zm',)): 19,
   (('S3',), ('Xp',)): 4,
   (('S3',), ('Xm',)): 25,
   (('S3',), ('Yp',)): 31,
   (('S3',), ('Ym',)): 4,
   (('S0',), ('Zm',)): 0},
  '001': {(('S0',), ('Zp',)): 29,
   (('S0',), ('Zm',)): 1,
   (('S0',), ('Xp',)): 30,
   (('S0',), ('Xm',)): 17,
   (('S0',), ('Yp',)): 7,
   (('S0',), ('Ym',)): 13,
   (('S1',), ('Zp',)): 13,
   (('S1',), ('Zm',)): 25,
   (('S1',), ('Xp',)): 26,
   (('S1',), ('Xm',)): 3,
   (('S1',), ('Yp',)): 27,
   (('S1',), ('Ym',)): 16,
   (('S2',), ('Zp',)):

In [63]:
organized_data[-1][-1]

{'10': {(('S0',), ('Zp',)): 4,
  (('S0',), ('Xp',)): 1,
  (('S0',), ('Xm',)): 2,
  (('S0',), ('Ym',)): 2,
  (('S1',), ('Zm',)): 2,
  (('S1',), ('Zp',)): 1,
  (('S1',), ('Xm',)): 1,
  (('S1',), ('Yp',)): 2,
  (('S2',), ('Zp',)): 1,
  (('S2',), ('Zm',)): 2,
  (('S2',), ('Xm',)): 2,
  (('S2',), ('Xp',)): 4,
  (('S2',), ('Yp',)): 3,
  (('S3',), ('Zm',)): 3,
  (('S3',), ('Xp',)): 2,
  (('S3',), ('Yp',)): 1,
  (('S0',), ('Zm',)): 0,
  (('S0',), ('Yp',)): 0,
  (('S1',), ('Xp',)): 0,
  (('S1',), ('Ym',)): 0,
  (('S2',), ('Ym',)): 0,
  (('S3',), ('Zp',)): 0,
  (('S3',), ('Xm',)): 0,
  (('S3',), ('Ym',)): 0},
 '01': {(('S0',), ('Zm',)): 3,
  (('S0',), ('Xm',)): 3,
  (('S0',), ('Yp',)): 1,
  (('S1',), ('Zm',)): 1,
  (('S1',), ('Zp',)): 3,
  (('S1',), ('Xp',)): 4,
  (('S1',), ('Yp',)): 3,
  (('S2',), ('Zm',)): 2,
  (('S2',), ('Xm',)): 2,
  (('S3',), ('Zp',)): 1,
  (('S3',), ('Xm',)): 4,
  (('S3',), ('Xp',)): 3,
  (('S3',), ('Ym',)): 3,
  (('S0',), ('Zp',)): 0,
  (('S0',), ('Xp',)): 0,
  (('S0',), 

In [81]:
recombined_lists = []
filtered_recombined_lists = []
norm_filtered_lists = []
for i in range(0, num_tomo_circuits):
    direct_models = ml.direct_fragment_model(organized_data[i])
    likely_models = ml.maximum_likelihood_model(direct_models)

    direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map_list[i])
    likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map_list[i])
    recombined_lists.append(likely_recombined_dist)
    filtered_recombined_lists.append(filter_results(likely_recombined_dist, [0]))
    norm_filtered_lists.append(norm_dict(filter_results(likely_recombined_dist, [0])))

{'000': {(('S0',), ('Zp',)): 4561, (('S0',), ('Zm',)): 56, (('S0',), ('Xp',)): 2623, (('S0',), ('Xm',)): 2073, (('S0',), ('Yp',)): 2311, (('S0',), ('Ym',)): 2324, (('S1',), ('Zp',)): 1598, (('S1',), ('Zm',)): 2799, (('S1',), ('Xp',)): 4200, (('S1',), ('Xm',)): 231, (('S1',), ('Yp',)): 2307, (('S1',), ('Ym',)): 2071, (('S2',), ('Zp',)): 1436, (('S2',), ('Zm',)): 2623, (('S2',), ('Xp',)): 1359, (('S2',), ('Xm',)): 2648, (('S2',), ('Yp',)): 357, (('S2',), ('Ym',)): 3641, (('S3',), ('Zp',)): 1725, (('S3',), ('Zm',)): 2654, (('S3',), ('Xp',)): 1010, (('S3',), ('Xm',)): 3258, (('S3',), ('Yp',)): 3821, (('S3',), ('Ym',)): 562}, '001': {(('S0',), ('Zp',)): 4355, (('S0',), ('Zm',)): 48, (('S0',), ('Xp',)): 2353, (('S0',), ('Xm',)): 2022, (('S0',), ('Yp',)): 1995, (('S0',), ('Ym',)): 2363, (('S1',), ('Zp',)): 1542, (('S1',), ('Zm',)): 3077, (('S1',), ('Xp',)): 4281, (('S1',), ('Xm',)): 304, (('S1',), ('Yp',)): 2370, (('S1',), ('Ym',)): 2272, (('S2',), ('Zp',)): 1409, (('S2',), ('Zm',)): 2358, ((

{'000': {(('S0',), ('Zp',)): 4606, (('S0',), ('Zm',)): 41, (('S0',), ('Xp',)): 2378, (('S0',), ('Xm',)): 2207, (('S0',), ('Yp',)): 2246, (('S0',), ('Ym',)): 2323, (('S1',), ('Zp',)): 1543, (('S1',), ('Zm',)): 2783, (('S1',), ('Xp',)): 4086, (('S1',), ('Xm',)): 249, (('S1',), ('Yp',)): 2167, (('S1',), ('Ym',)): 2124, (('S2',), ('Zp',)): 1557, (('S2',), ('Zm',)): 2371, (('S2',), ('Xp',)): 1191, (('S2',), ('Xm',)): 2733, (('S2',), ('Yp',)): 346, (('S2',), ('Ym',)): 3678, (('S3',), ('Zp',)): 1787, (('S3',), ('Zm',)): 2547, (('S3',), ('Xp',)): 1046, (('S3',), ('Xm',)): 3313, (('S3',), ('Yp',)): 3840, (('S3',), ('Ym',)): 570}, '001': {(('S0',), ('Zp',)): 4382, (('S0',), ('Zm',)): 69, (('S0',), ('Xp',)): 2542, (('S0',), ('Xm',)): 2028, (('S0',), ('Yp',)): 2190, (('S0',), ('Ym',)): 2309, (('S1',), ('Zp',)): 1616, (('S1',), ('Zm',)): 3066, (('S1',), ('Xp',)): 4267, (('S1',), ('Xm',)): 287, (('S1',), ('Yp',)): 2434, (('S1',), ('Ym',)): 2108, (('S2',), ('Zp',)): 1294, (('S2',), ('Zm',)): 2439, ((

{'000': {(('S0',), ('Zp',)): 8300, (('S0',), ('Zm',)): 86, (('S0',), ('Xp',)): 4626, (('S0',), ('Xm',)): 3666, (('S0',), ('Yp',)): 4020, (('S0',), ('Ym',)): 4275, (('S1',), ('Zp',)): 2982, (('S1',), ('Zm',)): 5556, (('S1',), ('Xp',)): 7963, (('S1',), ('Xm',)): 437, (('S1',), ('Yp',)): 4594, (('S1',), ('Ym',)): 3938, (('S2',), ('Zp',)): 3093, (('S2',), ('Zm',)): 5269, (('S2',), ('Xp',)): 2438, (('S2',), ('Xm',)): 5964, (('S2',), ('Yp',)): 879, (('S2',), ('Ym',)): 7568, (('S3',), ('Zp',)): 2977, (('S3',), ('Zm',)): 4484, (('S3',), ('Xp',)): 1745, (('S3',), ('Xm',)): 5691, (('S3',), ('Yp',)): 6493, (('S3',), ('Ym',)): 885}, '001': {(('S0',), ('Zp',)): 92, (('S0',), ('Zm',)): 4, (('S0',), ('Xp',)): 31, (('S0',), ('Xm',)): 33, (('S0',), ('Yp',)): 36, (('S0',), ('Ym',)): 49, (('S1',), ('Zp',)): 29, (('S1',), ('Zm',)): 49, (('S1',), ('Xp',)): 17, (('S1',), ('Xm',)): 80, (('S1',), ('Yp',)): 48, (('S1',), ('Ym',)): 34, (('S2',), ('Zp',)): 42, (('S2',), ('Zm',)): 48, (('S2',), ('Xp',)): 54, (('S

{'000': {(('S0',), ('Zp',)): 4129, (('S0',), ('Zm',)): 36, (('S0',), ('Xp',)): 2285, (('S0',), ('Xm',)): 1940, (('S0',), ('Yp',)): 1971, (('S0',), ('Ym',)): 2181, (('S1',), ('Zp',)): 1420, (('S1',), ('Zm',)): 2745, (('S1',), ('Xp',)): 3976, (('S1',), ('Xm',)): 250, (('S1',), ('Yp',)): 2078, (('S1',), ('Ym',)): 2034, (('S2',), ('Zp',)): 1712, (('S2',), ('Zm',)): 2555, (('S2',), ('Xp',)): 1331, (('S2',), ('Xm',)): 3005, (('S2',), ('Yp',)): 408, (('S2',), ('Ym',)): 3795, (('S3',), ('Zp',)): 1543, (('S3',), ('Zm',)): 2320, (('S3',), ('Xp',)): 975, (('S3',), ('Xm',)): 2922, (('S3',), ('Yp',)): 3525, (('S3',), ('Ym',)): 476}, '001': {(('S0',), ('Zp',)): 4104, (('S0',), ('Zm',)): 41, (('S0',), ('Xp',)): 2225, (('S0',), ('Xm',)): 1835, (('S0',), ('Yp',)): 2012, (('S0',), ('Ym',)): 2146, (('S1',), ('Zp',)): 1509, (('S1',), ('Zm',)): 2937, (('S1',), ('Xp',)): 4043, (('S1',), ('Xm',)): 289, (('S1',), ('Yp',)): 2468, (('S1',), ('Ym',)): 1840, (('S2',), ('Zp',)): 1481, (('S2',), ('Zm',)): 2780, (('

In [65]:
# direct_models = ml.direct_fragment_model(frag_data)
# likely_models = ml.maximum_likelihood_model(direct_models)

# direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map)
# likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map)

In [66]:
direct_recombined_dist

{'01000': 2.1402699201640574e-07,
 '10000': 9.230560949114871e-08,
 '00000': 0.002061238390843285,
 '11000': 1.4623514435048983e-05,
 '01001': 5.408491454288018e-06,
 '10001': 3.920687916198781e-06,
 '00001': 0.04997899111651147,
 '11001': 0.0010544539710131626,
 '01101': 2.07209020626837e-07,
 '10101': -3.231491784669421e-08,
 '00101': 0.0007417110382319844,
 '11101': -0.00010134115739504322,
 '01010': 3.866910125724818e-06,
 '10010': 2.289054557166949e-06,
 '00010': 0.035978354197824344,
 '11010': 3.64790121646499e-05,
 '01110': 7.880821900027064e-08,
 '10110': -1.1937408250561455e-08,
 '00110': 0.00033268211201734716,
 '11110': 2.0593623675638648e-05,
 '01011': 0.00010052437851120349,
 '10011': 6.613696069780429e-05,
 '00011': 0.8867287119664442,
 '11011': 0.01356129332571253,
 '01111': 2.3915915432348247e-06,
 '10111': -5.093838388468747e-07,
 '00111': 0.00960082361738667,
 '11111': -0.00020331772886865614,
 '01100': 1.0625840225968007e-10,
 '10100': -1.0007142453340554e-09,
 '0010

In [67]:
likely_recombined_dist

{'00000': 0.0015543152474964423,
 '11000': 6.239824033772249e-06,
 '00001': 0.04951443958923457,
 '11001': 0.0011292380569282439,
 '00101': 1.8025502299038464e-05,
 '11101': 2.4877697895548337e-07,
 '00010': 0.035293759175018445,
 '11010': 0.0004340563650717368,
 '00011': 0.8886716300234819,
 '11011': 0.014692427754247115,
 '00111': 0.008541596190878565,
 '11111': 0.00014402349433109633}

In [68]:
filtered_recombined_lists

[{'0000': 0.06858727611485013,
  '1000': 0.06989069003554157,
  '0100': 0.06533103871402818,
  '1100': 0.06642106545705295,
  '0001': 0.06494343369085769,
  '1001': 0.066210758192468,
  '0101': 0.06108313719144616,
  '1101': 0.06221375936549125,
  '0010': 0.058302160416993286,
  '1010': 0.059460966903576724,
  '0110': 0.05632321295111803,
  '1110': 0.0573801183558802,
  '0011': 0.05398251874958727,
  '1011': 0.05511236626120653,
  '0111': 0.05015357296196566,
  '1111': 0.0511769945769759},
 {'0000': 0.2736604956396639,
  '0100': 0.00046385280735129926,
  '0001': 0.2524006098740225,
  '0101': 0.00043035093057662593,
  '0010': 0.23078067855394938,
  '0110': 0.000585890601655554,
  '0011': 0.21242816894634353,
  '0111': 0.00015084698972461354},
 {'1000': 0.00013133782064274607,
  '0000': 0.00012830811747713324,
  '1100': 8.428197948191302e-05,
  '0100': 8.268171013866802e-05,
  '1001': 0.012125868129991568,
  '0001': 0.011927223874219094,
  '1101': 0.011864843609745932,
  '0101': 0.011645

In [69]:
from utils.utils import filter_results

In [70]:
filter_direct_recombined = filter_results(direct_recombined_dist, [0])

In [71]:
filter_direct_recombined

{'1000': 2.1402699201640574e-07,
 '0000': 0.002061238390843285,
 '1001': 5.408491454288018e-06,
 '0001': 0.04997899111651147,
 '1101': 2.07209020626837e-07,
 '0101': 0.0007417110382319844,
 '1010': 3.866910125724818e-06,
 '0010': 0.035978354197824344,
 '1110': 7.880821900027064e-08,
 '0110': 0.00033268211201734716,
 '1011': 0.00010052437851120349,
 '0011': 0.8867287119664442,
 '1111': 2.3915915432348247e-06,
 '0111': 0.00960082361738667,
 '1100': 1.0625840225968007e-10,
 '0100': 5.0151010259287185e-06}

In [72]:

filter_likely_recombined = filter_results(likely_recombined_dist, [0])

In [73]:
filter_likely_recombined

{'0000': 0.0015543152474964423,
 '0001': 0.04951443958923457,
 '0101': 1.8025502299038464e-05,
 '0010': 0.035293759175018445,
 '0011': 0.8886716300234819,
 '0111': 0.008541596190878565}

In [74]:
def norm_dict(dictionary):
    total = total_counts(dictionary)
    norm_dist = {}
    for i in dictionary.keys():
        norm_dist[i] = dictionary[i]/total
    return norm_dist

In [75]:
norm_filter_dist = norm_dict(filter_likely_recombined)

In [76]:
likely_recombined_dist['00011']

0.8886716300234819

In [77]:
#normalized distribution after mitigation: 0.9189375->0.9286796265616319
norm_filter_dist['0011']

0.9034945736620934

In [78]:
norm_filter_dist

{'0000': 0.0015802410524078305,
 '0001': 0.050340334917196444,
 '0101': 1.8326165666259096e-05,
 '0010': 0.03588245514028989,
 '0011': 0.9034945736620934,
 '0111': 0.00868406906234609}

In [82]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(norm_filtered_lists[i], shots = total_counts(norm_filtered_lists[i]), Pauli = Pauli_tuple[0])  

In [83]:
final_expect_val

(-1.0230049768496081+0j)

In [84]:
import json
with open('LiH_toronto_check3.json', 'w') as f:
    json.dump(norm_filtered_lists, f)